In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

data = pd.read_csv("/kaggle/input/first001/IMDB Dataset.csv", header=None)

/kaggle/input/test001/test.csv
/kaggle/input/first001/IMDB Dataset.csv


In [2]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

encoded_data = tokenizer.batch_encode_plus(
    data[0].tolist(),
    add_special_tokens=True,
    max_length=256,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

labels = data[1].tolist()
# Map sentiment labels to numerical values
label_map = {'positive': 1, 'negative': 0}  # Assuming 0 represents negative sentiment and 1 represents positive sentiment
labels = [label_map[label] for label in labels]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    encoded_data['input_ids'],
    labels,
    test_size=0.2,
    random_state=42
)

# Load the pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set up the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [4]:
# Define the training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3
batch_size = 16

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    
    model.train()
    for batch_start in range(0, len(train_inputs), batch_size):
        batch_end = min(batch_start + batch_size, len(train_inputs))
        batch_inputs = train_inputs[batch_start:batch_end].to(device)
        batch_labels = torch.tensor(train_labels[batch_start:batch_end]).to(device)
        
        outputs = model(batch_inputs, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    print(f"Training loss: {loss.item()}")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1/3
Training loss: 0.3709137737751007
Epoch 2/3
Training loss: 0.2070712298154831
Epoch 3/3
Training loss: 0.030927781015634537


In [5]:
# Save the trained model
model.save_pretrained("sentimental_part")
print("Model saved successfully.")

Model saved successfully.
